In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import timm

In [2]:
model = timm.create_model("tf_efficientnetv2_s.in1k", pretrained=True, num_classes=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

# ✅ Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(90),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [8]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.device_count() > 0 else "None")


CUDA available: True
Device count: 1
Current device: 0
Device name: NVIDIA TITAN V


In [3]:
from wholeslidedata.annotation.wholeslideannotation import WholeSlideAnnotation
import numpy as np
import cv2
import math

def rotate_point(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def find_tcga_rotation(dir_imgs, dir_XML, name):
    PNG_img = cv2.imread(dir_imgs+name)
    shapes = PNG_img.shape
    
    xml_info = WholeSlideAnnotation(dir_XML+name[:60]+'.xml')
    
    annotations = xml_info.sampling_annotations
    
    labels_values = xml_info.labels.values
    labels_names = xml_info.labels.names
    annotations = xml_info.sampling_annotations
    
    rois = []
    roi_label = labels_names.index('roi')
    roi_label = labels_values[roi_label]
    for i in annotations:        
        if i.label.value == roi_label:
            temp = i.wkt[10:-2].split(', ')        
            temp = [i.split(' ') for i in temp]
            temp = np.array([[int(float(j)) for j in i] for i in temp])
            rois.append(temp)
    
    roi = rois[0]
    shapes = PNG_img.shape
    move_x = (max(roi[:,0])+min(roi[:,0]))/2-shapes[1]/2
    move_y = (max(roi[:,1])+min(roi[:,1]))/2-shapes[0]/2
    roi[:,0] = roi[:,0]-move_x
    roi[:,1] = roi[:,1]-move_y

    for j in range(roi.shape[0]):
        if roi[j,0]==0 and roi[j,1]!=0:
            x1,y1 = roi[j,:]
        if roi[j,0]!=0 and roi[j,1]==0:
            x2,y2 = roi[j,:]    
    angle = np.degrees(np.arctan((y2-y1)/(x2-x1)))     
    angle_image = np.degrees(np.arctan((shapes[0]-0)/(shapes[1]-0)))  

    if abs(np.round(angle_image)) != abs(np.round(angle)):        
        if abs(angle)<45:
            angle = abs(angle)
        else:
            angle = -(90-abs(angle))  
        origin = [shapes[1]/2,shapes[0]/2]            
        rois_rotated = np.zeros((5,2))
        rois_rotated[0,:] = rotate_point(origin,roi[0,:],np.radians(angle))
        rois_rotated[1,:] = rotate_point(origin,roi[1,:],np.radians(angle))
        rois_rotated[2,:] = rotate_point(origin,roi[2,:],np.radians(angle))
        rois_rotated[3,:] = rotate_point(origin,roi[3,:],np.radians(angle))
        rois_rotated[4,:] = rois_rotated[0,:]
        
        x_min = int(min(rois_rotated[:,0]))
        x_max = int(max(rois_rotated[:,0]))
        y_min = int(min(rois_rotated[:,1]))
        y_max = int(max(rois_rotated[:,1]))  

        if x_min<0:
            x_min = 0
        if x_max>shapes[1]:
            x_max = shapes[1]
        if y_min<0:
            y_min = 0
        if y_max>shapes[0]:
            y_max = shapes[0]    

    else:
        angle = 0
        x_min = 0
        y_min = 0
        x_max = 0
        y_max = 0      
    return angle,x_min,x_max,y_min,y_max

In [5]:
import os
import cv2
from wholeslidedata.image.wholeslideimage import WholeSlideImage
import numpy as np

####################################################
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

def extract_tcga_data(data_path):
    dir_TIFF_images_WSIROIS = data_path+'/wsirois/wsi-level-annotations/images/'
    dir_PNG_masks = data_path+'/wsirois/roi-level-annotations/tissue-bcss/masks/'    
    dir_PNG_images = data_path+'/wsirois/roi-level-annotations/tissue-bcss/images/'
    dir_XML = data_path+'/wsirois/wsi-level-annotations/annotations-tissue-bcss-xmls/'    
   
    imgs_names = os.listdir(dir_TIFF_images_WSIROIS)
    imgs_names.sort()
    msks_names = os.listdir(dir_PNG_masks)
    msks_names.sort()
    imgs_names = [i for i in imgs_names if i.startswith('TCGA')]    
    png_names = os.listdir(dir_PNG_images)
    png_names.sort()
    png_names = png_names[:-1]    
    
    imgs_msks_names_masks_coorindates = []
    for i in range(len(imgs_names)):
        temp = imgs_names[i][:-4]
        msk_names = [j for j in msks_names if temp in j[:60]]
        msk_coordinates = [j[62:-5] for j in msk_names]
        msk_coordinates = [j.split(', ') for j in msk_coordinates]
        msk_coordinates = [[int(k) for k in j] for j in msk_coordinates]        
        imgs_msks_names_masks_coorindates.append([imgs_names[i], msk_names, msk_coordinates])  
    del temp, i, msk_names,msk_coordinates

    imgs = []
    msks = []
    for i in range(len(imgs_msks_names_masks_coorindates)):      
        tif_name = imgs_msks_names_masks_coorindates[i][0]
        tif_img = WholeSlideImage(dir_TIFF_images_WSIROIS+tif_name)
        print(tif_name)        
        msk_name = imgs_msks_names_masks_coorindates[i][1][0]
        x_min_bound = imgs_msks_names_masks_coorindates[i][2][0][0]
        y_min_bound = imgs_msks_names_masks_coorindates[i][2][0][1]
        x_max_bound = imgs_msks_names_masks_coorindates[i][2][0][2]
        y_max_bound = imgs_msks_names_masks_coorindates[i][2][0][3]
        img = tif_img.get_patch(x_min_bound, y_min_bound,x_max_bound-x_min_bound, y_max_bound-y_min_bound, tif_img.spacings[0], center = False)
        msk = cv2.imread(dir_PNG_masks+msk_name)[:,:,0]
        angle,x_min,x_max,y_min,y_max = find_tcga_rotation(dir_PNG_images, dir_XML, msk_name)
        if angle !=0:
            img_rotated = rotate_image(img, -angle) 
            img_rotated = img_rotated[y_min:y_max,x_min:x_max]
            msk_rotated = rotate_image(msk, -angle) 
            msk_rotated = msk_rotated[y_min:y_max,x_min:x_max]
        else:
            img_rotated = img
            msk_rotated = msk
        imgs.append(img_rotated)
        msks.append(msk_rotated) 
        
    return imgs, msks

In [6]:
def change_masks(input_data):    
    # This function relables the input_mask to segment the tissue into "rest", "tumor", and "stroma" classes.
    # input_data: numpy array of size (N,M)
    input_data[input_data == 3] = 0
    input_data[input_data == 4] = 0
    input_data[input_data == 5] = 0
    input_data[input_data == 6] = 2
    input_data[input_data == 7] = 0
    return input_data

In [8]:
data_path = '/lab/deasylab3/Jung/tiger/'
##### Loading TCGA images and masks #######

imgs_1, msks_1 = extract_tcga_data(data_path)

TCGA-A1-A0SK-01Z-00-DX1.A44D70FA-4D96-43F4-9DD7-A61535786297.tif


AttributeError: 'PolygonAnnotation' object has no attribute 'wkt'